<a href="https://colab.research.google.com/github/OVP2023/NN/blob/main/Digit_Recognizer_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Digit Recognizer
#https://www.kaggle.com/c/digit-recognizer


import cv2
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf


train = np.loadtxt('/content/drive/MyDrive/Datasets/CV/digit/train.csv', delimiter=',', skiprows=1)
test = np.loadtxt('/content/drive/MyDrive/Datasets/CV/digit/test.csv', delimiter=',', skiprows=1)

# сохраняем разметку в отдельную переменную
train_label = train[:, 0]
# приводим размерность к удобному для обаботки виду
train_img = np.resize(train[:, 1:], (train.shape[0], 28, 28))


In [ ]:
train_img.shape

(42000, 28, 28)

In [ ]:
# сохраняем разметку в отдельную переменную
train_label = train[:, 0]

# приводим размерность к удобному для обаботки виду
# добавляем размерность канала
train_img = np.resize(train[:, 1:], (train.shape[0], 28, 28, 1))
test_img = np.resize(test, (test.shape[0], 28, 28, 1))

In [ ]:
#Визуализируем исходные данные

fig = plt.figure(figsize=(20, 10))
for i, img in enumerate(train_img[0:5, :], 1):
    subplot = fig.add_subplot(1, 5, i)
    plt.imshow(img[:,:,0], cmap='gray');
    subplot.set_title('%s' % train_label[i - 1]);

In [ ]:
#Собираем сверточную сеть для обучения
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=6,
                           kernel_size=(5, 5),
                           padding='same',
                           activation='relu',
                           input_shape=x_train.shape[1:]),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='valid'),

    tf.keras.layers.Conv2D(filters=16,
                           kernel_size=(5, 5),
                           padding='same',
                           activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='valid'),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
One-hot кодировка разметки
y_train_labels = tf.keras.utils.to_categorical(y_train)print(y_train.shape)
print(y_train_labels.shape)

In [ ]:
#Запускаем обучение

model.fit(x_train,
          y_train_labels,
          batch_size=32,
          epochs=5,
          validation_split=0.2)

In [ ]:
#Предсказываем класс объекта

y_pred = model.predict(x_val)
y_pred_labels = np.argmax(y_pred, axis=1)

In [ ]:
#Оцениваем качество решение на валидационной выборке

from sklearn.metrics import accuracy_score
print('Accuracy: %s' % accuracy_score(y_val, y_pred_labels))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred_labels))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred_labels))

In [ ]:
#Визуализируем предсказания для тестовой выборки

y_pred_test = model.predict(test_img)
y_pred_test_labels = np.argmax(y_pred_test, axis=1)

In [ ]:
fig = plt.figure(figsize=(20, 10))
indices = np.random.choice(range(len(test_img)), 5)
img_prediction = zip(test_img[indices], y_pred_test_labels[indices])
for i, (img, pred) in enumerate(img_prediction, 1):
    subplot = fig.add_subplot(1, 5, i)
    plt.imshow(img[...,0], cmap='gray');
    subplot.set_title('%d' % pred);

In [ ]:
#Готовим файл для отправки

with open('submit.csv', 'w') as dst:
    dst.write('ImageId,Label\n')
    for i, p in enumerate(y_pred_test_labels, 1):
        dst.write('%s,%d\n' % (i, p))

In [ ]:
!head submit.csv